## SVM with HOG features
Another approach to detect faces is using a Histogram of Oriented Gradients (HOG) in combination with a support vector machine as classifier.
HOG is a feature descriptor and is commonly used in image processing. HOG is an algorithm that typically consists of the following steps:
1. Image Preprocessing
2. Calculate Gradient
3. Create Histogram of Oriented Gradients
4. Normalise Histogram Vectors

In the following, we will give a brief overview of the steps in HOG.

### HOG Steps

#### Step 1 - Image Preprocessing

To divide the image into blocks later on, the image is first transformed into one with a width/height-ratio of 1:2 (commonly 64x128). Afterwards, the image is divided into blocks (commonly 8x8) of pixels. For the case of an 64x128 image, the resulting image would be a 8x16 grid with 8x8 blocks. In the following, we will assume a block size of 8x8 and an image size of 64x128 to simplify the explanation. 

#### Step 2 - Calculate Gradient

For each of these 8x8 pixels, the gradient vector is calculated. Therefore, we have to calculate the gradient in the x and y-direction. For a given pixel at the position (x,y) on an image I, the gradient in x direction is calculated as follows:
$$G_x = I(x+1, y) - I(x-1, y)$$
and the gradient in y-direction as:
$$G_y = I(x, y+1) - I(x, y-1)$$

![Alt text](images/Hog8x8Grid.png)

For the pixel 60 on the image, the gradient in x-direction will be:
$$G_x = I(x+1, y) - I(x-1, y) = 70 - 40 = 30$$

and for the y-direction:
$$G_y = I(x, y+1) - I(x, y-1) = 70 - 20 = 50$$

Using the gradient in x and y direction, the magnitude and direction of the gradient vector will be calculated using:
$$\text{magnitude} = \sqrt{G_x^2 + G_y^2}$$
$$\text{direction} = \arctan\left(\frac{G_y}{G_x}\right)$$

For the example the magnitude would be:
$$\sqrt{G_x^2 + G_y^2} = \sqrt{30^2 + 50^2} \approx 58.31 $$

and the direction would be:
$$\arctan\left(\frac{50}{30}\right) \approx 59.04 \degree $$

This processed is repeated for every pixel in the 8x8 block so that we receive one 8x8 matrix with the direction of the gradient vector and one 8x8 matrix with the magnitude of the gradient vector for the corresponding pixels.

#### Step 3 - Create Histogram of Oriented Gradients

In the next step, the Histogram of Oriented Gradients is created with the 8x8 matrix of the direction of the gradient vector and the 8x8 matrix of the magnitude of the gradient vector.

![Alt text](images/gradient-histogram.png) TODO: Change the image. Currently the classes are not perfectly represented!

Typically, the direction values are normalised to values between 0 and 180 degrees and are used on the x-axis. On the y-axis a running sum of the magnitude values for a given direction is displayed. For a given direction of the gradient vector of a pixel, the corresponding magnitude is added to the bin on the x-axis. For instance, if the gradient vector of a given pixel has a direction of 20 and a magnitude of 60, the value 60 is added to the running sum on the y-axis. If the direction is not exactly in one of these bins, the magnitude has to be split up and placed in the two closest classes. For instance, if the gradient vector of a given pixel has a direction of 30 and a magnitude of 60, then 30 has to be added to the 20 class and 30 has to be added to the 40 class.

For each of these 8x8 Blocks, a Histogram of Oriented Gradients with 9 entries will be received. These 9 entries, which are the 9 bins on the x-axis of the histogram, can be used in a 9x1 vector for the next step.

#### Step 4 - Normalise Histogram Vectors

To enhance the robustness, make the descriptor invariant to changes in illumination, and to improve the classification performance, the resulting 9x1 vectors for each of these 8x8 blocks will have to be normalised. A common way to do that is to use 4 of these 8x8 blocks, which forms one 16x16 block. This 16x16 block consists of four 9x1 vectors, which can be linearised to one 36x1 vector. This vector is then normalised using the following formulas:

$$\text{magnitude} = \sqrt{b_1^2 + b_2^2 + ..... + b_{36}^2}$$
$$\text{normalised vector} = [\frac{b_1}{\text{magnitude}}, \frac{b_2}{\text{magnitude}}, ....., \frac{b_{36}}{\text{magnitude}}]$$

As a result, we receive a normalised 36x1 vector that describes the feature of a 16x16 block. A 64x128 image has 7x15 of these 16x16 blocks, so in total we have 7x15x36x1 = 3780 entries, that are often placed in a 1x3780 vector. This vector probably has to be reduced (to prevent overfitting) using for instance PCA (Principal Component Analysis). However, this will not be explained in this article. We assume that the vector will be fed to an SVM in the following.

### SVM with HOG features

The resulting vector from the HOG algorithm, that was potentially reduced using PCA, is often fed to an SVM. The SVM tries to find a hyperplane that best separates the datapoints of different classes in a high-dimensional space. On a basic level, the datapoints can be classified into images that contain a face (positive samples), and images that don't contain a face (negative samples). The HOG features extracted from negative and positive samples can then be used to train the SVM so that it learns to distinguish between images that contain faces and ones that don't. Additionally, a trained SVM can be used as a sliding window that analyses a small part of a predefined size of the image to determine whether this part contains a face or not. This allows to not only classify images with faces correctly, but also to detect faces on the image.

